### People Car cl

In [ ]:
!CUDA_VISIBEL_DEVICES=0 \
python -m torch.distributed.launch --nproc_per_node=1 \
CVPR/train_people_car_cl.py \
--csv-dir data/train_val_cl_20fold.csv \
--config-name 'config_eva_vit_people_car_cl' \
--image-size 224 \
--epochs 11 \
--init-lr 3e-5 \
--batch-size 32 \
--num-workers 8 \
--nbatch_log 300 \
--warmup_epochs 0 \
--fold 1

### People+Car

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
df1 = pd.read_csv('output/test_people_out.csv')
df2 = pd.read_csv('output/test_car_out.csv')
data = {
    'results':[]
}
for idx, row in tqdm(df2.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append(row[f'img_{i}'].split('/')[-1])
    data['results'].append(res)
for idx, row in tqdm(df1.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append(row[f'img_{i}'].split('/')[-1])
    data['results'].append(res)

with open('output/submission.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### people only

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
df1 = pd.read_csv('output/test_people_out.csv')
df2 = pd.read_csv('output/test_car_out.csv')
data = {
    'results':[]
}
for idx, row in tqdm(df2.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append('nothing.jpg')
    data['results'].append(res)
for idx, row in tqdm(df1.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append(row[f'img_{i}'].split('/')[-1])
    data['results'].append(res)

with open('output/submission.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

### Car only

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
df1 = pd.read_csv('output/test_people_out.csv')
df2 = pd.read_csv('output/test_car_out.csv')
data = {
    'results':[]
}
for idx, row in tqdm(df2.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append(row[f'img_{i}'].split('/')[-1])
    data['results'].append(res)
for idx, row in tqdm(df1.iterrows(), total=len(df1)):
    res = {'text':row.prompt+'.', 'image_names':[]}
    for i in range(10):
        res['image_names'].append('nothing.jpg')
    data['results'].append(res)

with open('output/submission.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import torch
dicts = torch.load('pretrained_weights/EVA02_CLIP_L_336_psz14_s6B_vision.pt')
new_dicts = {}
decoup_stage = 22
for k, v in dicts.items():
    if 'blocks.' in k:
        stage = int(k.split('.')[1])
        if stage < decoup_stage:
            new_dicts[k.replace('blocks.', 'shared_blocks.')] = v
        else:
            for i in range(12):
                new_dicts[f'decoupler.{i}.'+k.replace(str(stage), str(stage-decoup_stage))] = v
    elif 'norm' in k or 'head' in k:
        for i in range(12):
            new_dicts[f'decoupler.{i}.'+k] = v
    else:
        new_dicts[k] = v
torch.save(new_dicts, f'pretrained_weights/EVA02_CLIP_L_336_psz14_s6B_vision_decoupler_{decoup_stage}.pt')

In [ ]:
import torch
dicts = torch.load('pretrained_weights/EVA02_CLIP_L_psz14_s4B_vision.pt')
new_dicts = {}
decoup_stage = 22
for k, v in dicts.items():
    if 'blocks.' in k:
        stage = int(k.split('.')[1])
        if stage < decoup_stage:
            new_dicts[k.replace('blocks.', 'shared_blocks.')] = v
        else:
            for i in range(12):
                new_dicts[f'decoupler.{i}.'+k.replace(str(stage), str(stage-decoup_stage))] = v
    elif 'norm' in k or 'head' in k:
        for i in range(12):
            new_dicts[f'decoupler.{i}.'+k] = v
    else:
        new_dicts[k] = v
torch.save(new_dicts, f'pretrained_weights/EVA02_CLIP_L_psz14_s4B_vision_decoupler_{decoup_stage}.pt')